Version 1.0.1, Python 2.7.14, conda 4.3.34

In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
#import lightgbm as lgb
#from catboost import CatBoostRegressor
#from multiprocessing import *

#import sys

from sklearn.metrics import mean_squared_error
from prepare_data1 import k_fold_mean, loo_mean, smoothing_mean, expanding_mean
from prepare_data1 import make_grid, get_target, join_to_existing, prepare_test
from prepare_data1 import count_categories, create_df

import matplotlib.pyplot as plt
%matplotlib inline

for p in [np, pd, xgb]:
    print (p.__name__, p.__version__)

('numpy', '1.14.2')
('pandas', u'0.22.0')
('xgboost', '0.7.post3')


Load the data:

In [2]:
from prepare_data import read_data
sales, items, categories, shops, test = read_data()
print(test.shape)

(214200, 3)


Create a data frame with all the informations given:

In [3]:
df = create_df(sales, items, categories, shops) 
df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name,year,month
0,02.01.2013,0,59,22154,999.0,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",2013,1
1,26.04.2013,3,59,944,150.0,1.0,2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",2013,4
2,26.06.2013,5,59,944,199.5,1.0,2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",2013,6
3,20.07.2013,6,59,944,199.5,1.0,2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",2013,7
4,14.09.2013,8,59,944,299.0,2.0,2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",2013,9


In [20]:
index_3 = ['shop_id', 'item_id', 'date_block_num']

grid = make_grid(sales, index_3)
target = get_target(sales, index_3)
all_data = join_to_existing(grid, target, index_3, 0)

In [7]:
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

all_data = all_data.assign(month = (all_data.date_block_num%12)+1)
all_data = all_data.assign(year = 2013)
all_data['year'] = np.where(all_data.date_block_num < 12, 2013, 2014)
all_data['year'] = np.where(all_data.date_block_num > 23, 2015, all_data.year)


categories = df[['shop_id', 'item_id', 'date_block_num', 'item_category_id']]
category = count_categories(categories, index_3)
all_data = join_to_existing(all_data, category, index_3, 0)


#make function of this one
target_mean = all_data[['shop_id','item_id','target']].groupby(['shop_id','item_id'], as_index=False)[['target']].mean()
target_mean = target_mean.rename(columns={'target':'target_mean'})


index_cols = ['shop_id', 'item_id']
all_data = join_to_existing(all_data, target_mean, index_cols, 0)
all_data = all_data.astype('float32')

In [8]:
all_data, target_kf  = k_fold_mean(all_data, 'target', 0.3343)
all_data = all_data.rename(columns={'new_name':'target_kf'})
all_data, target_loo = loo_mean(all_data, 'target', 0.3343)
all_data = all_data.rename(columns={'new_name':'target_loo'})
all_data, target_sm = smoothing_mean(all_data, 'target', 0.3343)
all_data = all_data.rename(columns={'new_name':'target_sm'})
all_data, target_exp = expanding_mean(all_data, 'target', 0.3343)
all_data = all_data.rename(columns={'new_name':'target_exp'})

Add more features:

In [9]:
all_data, category_kf = k_fold_mean(all_data, 'count_category', 0)
all_data = all_data.rename(columns={'new_name':'category_kf'})
all_data, category_loo = loo_mean(all_data, 'count_category', 0)
all_data = all_data.rename(columns={'new_name':'category_loo'})
all_data, category_sm = smoothing_mean(all_data, 'count_category', 0)
all_data = all_data.rename(columns={'new_name':'category_sm'})
all_data, category_exp = expanding_mean(all_data, 'count_category', 0)
all_data = all_data.rename(columns={'new_name':'category_exp'})
all_data.head()

,shop_id,item_id,date_block_num,target,month,year,count_category,target_mean,target_kf,target_loo,target_sm,target_exp,category_kf,category_loo,category_sm,category_exp
0,0.0,19.0,0.0,0.0,1.0,2013.0,0.0,0.0,0.334300,0.022727,0.237448,0.3343,0.000000,0.022727,0.006897,0.0
1,0.0,27.0,0.0,0.0,1.0,2013.0,0.0,0.0,0.048523,0.056911,0.089905,0.3343,0.048523,0.056911,0.050060,0.0
2,0.0,28.0,0.0,0.0,1.0,2013.0,0.0,0.0,0.142424,0.141414,0.168964,0.3343,0.142424,0.144781,0.123741,0.0
3,0.0,29.0,0.0,0.0,1.0,2013.0,0.0,0.0,0.030303,0.037500,0.107910,0.3343,0.030303,0.043750,0.033254,0.0
4,0.0,32.0,0.0,6.0,1.0,2013.0,4.0,8.0,0.894020,1.316088,1.260635,0.3343,0.847843,1.184858,1.117438,0.0


Create training data:

In [11]:
y = all_data.target.astype('int32')
X = all_data.drop(columns=['target'])
print(y.shape)
print(X.shape)
X.head()
X.info()

(10913850,)
(10913850, 15)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10913850 entries, 0 to 10913849
Data columns (total 15 columns):
shop_id           float32
item_id           float32
date_block_num    float32
month             float32
year              float32
count_category    float32
target_mean       float32
target_kf         float32
target_loo        float32
target_sm         float32
target_exp        float32
category_kf       float32
category_loo      float32
category_sm       float32
category_exp      float32
dtypes: float32(15)
memory usage: 707.8 MB


Create test data with same features as train data:

In [22]:
X_test = prepare_test(test)
X_test = join_to_existing(X_test, category, index_3, 0)
X_test = join_to_existing(X_test, target_mean, index_cols, 0.3343)
X_test.head()

,shop_id,item_id,date_block_num,month,year,count_category
0,5,5037,34,11,2015,0.0
1,5,5320,34,11,2015,0.0
2,5,5233,34,11,2015,0.0
3,5,5232,34,11,2015,0.0
4,5,5268,34,11,2015,0.0


check the merging, maybe there is a better solution, without writing all the indices.

In [25]:
X_kf = X_test.merge(target_kf, on=index_cols)
X_kf = X_kf.rename(columns={'new_name':'target_kf'})
index = ['shop_id', 'item_id', 'date_block_num', 'month', 'year', 'count_category', 'target_mean']
X_kf = X_kf.groupby(index, as_index=False)[['target_kf']].min()#try also with min
X_test = join_to_existing(X_test, X_kf, index, 0.3343)

X_loo = X_test.merge(target_loo, on=index_cols)
X_loo = X_loo.rename(columns={'new_name':'target_loo'})
index = ['shop_id', 'item_id', 'date_block_num', 'month', 'year', 'count_category', 'target_mean', 'target_kf']
X_loo = X_loo.groupby(index, as_index=False)[['target_loo']].min()#try also with min
X_test = join_to_existing(X_test, X_loo, index, 0.3343)

X_sm = X_test.merge(target_sm, on=index_cols)
X_sm = X_sm.rename(columns={'new_name':'target_sm'})
index = ['shop_id', 'item_id', 'date_block_num', 'month', 'year', 'count_category', 'target_mean', 'target_kf', 'target_loo']
X_sm = X_sm.groupby(index, as_index=False)[['target_sm']].min()#try also with min
X_test = join_to_existing(X_test, X_sm, index, 0.3343)

X_exp = X_test.merge(target_exp, on=index_cols)
X_exp = X_exp.rename(columns={'new_name':'target_exp'})
index = ['shop_id', 'item_id', 'date_block_num', 'month', 'year', 'count_category', 'target_mean', 'target_kf', 'target_loo', 'target_sm']
X_exp = X_exp.groupby(index, as_index=False)[['target_exp']].min()#try also with min
X_test = join_to_existing(X_test, X_exp, index, 0.3343)


X_kf = X_test.merge(category_kf, on=index_cols)
X_kf = X_kf.rename(columns={'new_name':'category_kf'})
index = ['shop_id', 'item_id', 'date_block_num', 'month', 'year', 'count_category', 'target_mean', 'target_kf', 'target_loo', 'target_sm', 'target_exp']
X_kf = X_kf.groupby(index, as_index=False)[['category_kf']].mean()#try also with min
X_test = join_to_existing(X_test, X_kf, index, 0)

X_loo = X_test.merge(category_loo, on=index_cols)
X_loo = X_loo.rename(columns={'new_name':'category_loo'})
index = ['shop_id', 'item_id', 'date_block_num', 'month', 'year', 'count_category', 'target_mean', 'target_kf', 'target_loo', 'target_sm', 'target_exp','category_kf']
X_loo = X_loo.groupby(index, as_index=False)[['category_loo']].mean()#try also with min
X_test = join_to_existing(X_test, X_loo, index, 0)

X_sm = X_test.merge(category_sm, on=index_cols)
X_sm = X_sm.rename(columns={'new_name':'category_sm'})
index = ['shop_id', 'item_id', 'date_block_num', 'month', 'year', 'count_category', 'target_mean', 'target_kf', 'target_loo', 'target_sm', 'target_exp','category_kf', 'category_loo']
X_sm = X_sm.groupby(index, as_index=False)[['category_sm']].mean()#try also with min
X_test = join_to_existing(X_test, X_sm, index, 0)

X_exp = X_test.merge(category_exp, on=index_cols)
X_exp = X_exp.rename(columns={'new_name':'category_exp'})
index = ['shop_id', 'item_id', 'date_block_num', 'month', 'year', 'count_category', 'target_mean', 'target_kf', 'target_loo', 'target_sm','target_exp','category_kf', 'category_loo', 'category_sm']
X_exp = X_exp.groupby(index, as_index=False)[['category_exp']].mean()#try also with min
X_test = join_to_existing(X_test, X_exp, index, 0)


X_test = X_test.astype('float32')
X_test.shape

(214200, 15)

Create validation set:

In [26]:
X_train = X[X.date_block_num < 33]
X_valid = X[X.date_block_num == 33]
y_train = y[:len(X_train)]
y_valid = y[len(X_train):]

In [ ]:
#idx_item1 = df_train.item_price[df.item_price <= 0]
#idx_item2 = df_train.item_price[df.item_price > 100000]

#df_train.drop(df_train.index[idx_item2.index[0]], inplace=True)
#df_train.item_price.replace(to_replace=-1, value=2499.0 , inplace=True) 

In [ ]:
#df_train.drop(columns=['item_name', 'shop_name', 'date', 'item_category_name', 'day'], inplace=True)

In [ ]:
#print(df_train.item_price.min())
#print(df_train.item_price.max())

In [ ]:
#df_train.sort_values(by=['date_block_num'], axis=0, inplace=True)
#df_train.head()

In [ ]:
#df_train=df_train.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False).item_cnt_day.sum()
#df_monthly=df_monthly.rename(columns={'item_cnt_day': 'item_cnt_month'}).astype('int32')
#df_monthly=df_monthly.reset_index().astype('int32')

In [ ]:
#prev_month = df_monthly
#df_monthly = df_monthly

In [ ]:
#monthly_sales=sales.groupby(["date_block_num","shop_id","item_id"])[
#    "date","item_price","item_cnt_day"].agg({"date":["min",'max'],"item_price":"mean","item_cnt_day":"sum"})

So, ready to train and validate, i.e. separate train data into train and validation set and train with xgboost or sth. else. Then prepare test data.

In [27]:
#XGBoost
def xgb_rmse(preds, y):
    y = y.get_label()
    score = np.sqrt(mean_squared_error(y.clip(0.,20.), preds.clip(0.,20.)))
    return 'RMSE', score

In [28]:
params = {'eta': 0.2, 'max_depth': 2, 'objective': 'reg:linear', 'eval_metric': 'rmse', 'seed': 18, 'silent': True}

watchlist = [(xgb.DMatrix(X_train, y_train), 'train'), (xgb.DMatrix(X_valid, y_valid), 'valid')]
xgb_model = xgb.train(params, xgb.DMatrix(X_train, y_train), 100,  watchlist, verbose_eval=10, feval=xgb_rmse, maximize=False, early_stopping_rounds=20)


[0]	train-rmse:3.06838	valid-rmse:5.21173	train-RMSE:1.07926	valid-RMSE:1.04805
Multiple eval metrics have been passed: 'valid-RMSE' will be used for early stopping.

Will train until valid-RMSE hasn't improved in 20 rounds.
[10]	train-rmse:2.10324	valid-rmse:4.77915	train-RMSE:0.436589	valid-RMSE:0.550111
[20]	train-rmse:1.98174	valid-rmse:4.66571	train-RMSE:0.35391	valid-RMSE:0.462459
[30]	train-rmse:1.94983	valid-rmse:4.6598	train-RMSE:0.351589	valid-RMSE:0.458056
[40]	train-rmse:1.93618	valid-rmse:4.63453	train-RMSE:0.349867	valid-RMSE:0.448821
[50]	train-rmse:1.91142	valid-rmse:4.59427	train-RMSE:0.349661	valid-RMSE:0.478965
[60]	train-rmse:1.89619	valid-rmse:4.58972	train-RMSE:0.349298	valid-RMSE:0.478649
Stopping. Best iteration:
[40]	train-rmse:1.93618	valid-rmse:4.63453	train-RMSE:0.349867	valid-RMSE:0.448821



In [29]:
test['item_cnt_month'] = xgb_model.predict(xgb.DMatrix(X_test), ntree_limit=xgb_model.best_ntree_limit)
test[['ID','item_cnt_month']].to_csv('xgb_submission.csv', index=False)